# Pilotage basique

Dans ce notebook, nous allons découvrir les commandes de base permettant de piloter le JetBot.

### Importation de la classe Robot

Le JetBot peut être piloté à l'aide des fonctions contenues dans la classe ``Robot``. Pour l'utiliser, nous avons tout d'abord besoin de l'importer sous Python.

In [1]:
from jetbot import Robot

Une fois la classe importée, nous pouvons initialiser une instance de type Robot :

In [2]:
robot = Robot()

### Commande du robot

Nous avons maintenant instancié une classe de type Robot nommée ``robot``. Cette instance nous permet d'utiliser des fonctions afin de contrôler le robot.

Par exemple, pour faire tourner la roue gauche du robot dans le sens des aiguilles d'une montre et à une vitesse correspondant à 30% de la vitesse maximale, il suffit d'appeler la méthode ``left`` :

In [5]:
robot.left(speed=0.3)

La roue de votre robot devrait tourner dans le sens des aiguilles d'une montre.

Maintenant, nous pouvons arrêter le robot avec la méthode ``stop`` :

In [6]:
robot.stop()

Si on souhaite faire tourner la roue du robot seulement pendant une période de temps spécifique, il suffit d'utiliser la méthode ``sleep`` contenue dans le package ``time``. On commence donc par importer ce package :

In [8]:
import time

Et ensuite, nous pouvons appeler la méthode ``sleep`` en donnant la durée d'exécution en seconde. L'exemple ci-dessous va faire tourner la roue pendant 0,5s :

In [9]:
robot.left(0.3)
time.sleep(0.5)
robot.stop()

La classe ``Robot`` possède bien sûr d'autres méthodes ! Par exemple : ``right`` (tourner la roue droite), ``forward`` (avancer), et ``backward`` (reculer).

Essayez par exemple de faire avancer le robot à 50% de sa vitesse maximale pendant 1 seconde ...

In [10]:
# Programme à compléter
# Faire avancer le robot en ligne droite, pendant 1s...



### Contrôle des moteurs individuellement

Si vous souhaitez activer les moteurs de manière individuelle, il existe deux manière de faire cela.

La première méthode utilise la méthode ``set_motors``. Par exemple, pour faire tourner le moteur gauche à 30% et le droit à 60%, on peut utiliser la commande suivante. De cette manière, le robot va dessiner un arc de cercle :

In [15]:
robot.set_motors(0.3, 0.6)
time.sleep(1.0)
robot.stop()

L'autre manière de faire est d'utiliser les attributs ``left_motor`` et ``right_motor``. Ils représentent les valeurs des vitesses de chaque moteur.

Ces attributs appartiennent à la classe ``Motor``, chacun possède un attribut nommé ``value``. C'est à cet attribut qu'il faut donner la valeur de la vitesse souhaitée :

In [16]:
robot.left_motor.value = 0.3
robot.right_motor.value = 0.6
time.sleep(1.0)
robot.left_motor.value = 0.0
robot.right_motor.value = 0.0

Vous devriez voir le robot exécuter la même trajectoire que précédemment.

### Lier les moteurs à des traitlets

Les [traitlets](https://github.com/ipython/traitlets) permettent de gérer des méthodes de manière dynamique en Python. Les attributs que nous avons vus juste avant sont en fait des traitlets.

Ce qui est sympa avec les traitlets, c'est qu'on peut les lier les uns aux autres... Cela nous permet de créer des interfaces graphiques sous les notebook de jupyter en liant ces traitlets avec des ``widgets``. On peut par exemple comme cela visualiser la vitesse d'un moteur...

Bon... le mieux est quand même de voir à quoi cela ressemble ! On va donc créer deux slides permettant de contrôler la vitesse des moteurs :

In [17]:
import ipywidgets.widgets as widgets
from IPython.display import display

# Création de deux slides avec un intervalle de [-1.0, 1.0]
slide_gauche = widgets.FloatSlider(description='gauche', min=-1.0, max=1.0, step=0.01, orientation='vertical')
slide_droit = widgets.FloatSlider(description='droit', min=-1.0, max=1.0, step=0.01, orientation='vertical')

# Création d'une box horizontale qui va englober ces slides
box_slides = widgets.HBox([slide_gauche, slide_droit])

# Affiche la boite contenant les slides
display(box_slides)

Vous devriez voir deux slides verticaux.

Essayez de faire varier la position des slides... Vous remarquez que rien ne se passe sur le robot. C'est parce que nous n'avons pas connecté les slides aux moteurs !

Pour les connecter, nous allons utiliser la fonction ``link`` du package traitlets. Nous allons :
- Connecter la valeur donnée au widget "slide_gauche" à la valeur left_motor
- Faire de même pour le widget "slide_droit"

In [18]:
import traitlets

lien_gauche = traitlets.link((slide_gauche, 'value'), (robot.left_motor, 'value'))
lien_droit = traitlets.link((slide_droit, 'value'), (robot.right_motor, 'value'))

Maintenant, si vous bougez de nouveau les slides... Les moteurs vont tourner !

La fonction ``link`` que nous avons utilisée est un lien bidirectionnel. Cela signifie que si nous modifions la valeur du moteur (robot.left_motor ou robot.right_motor), les slides vont changer !

Essayez par exemple le code ci-dessous. Vous verrez les roues du robot tourner, mais également les slides bouger !

In [20]:
robot.forward(0.3)
time.sleep(1.0)
robot.stop()

Si nous souhaitons annuler le lien, il suffit d'utiliser la méthode ``unlink`` sur chaque lien :

In [ ]:
lien_gauche.unlink()
lien_droit.unlink()

On peut aussi créer des liens qui ne soient pas *bidirectionnels*. Par exemple, cela peut permettre d'utiliser les slides uniquement pour visualiser la valeur des moteurs mais sans pouvoir les commander avec eux.

Pour cela, il faut utiliser la fonction ``dlink``. L'attribut de gauche est la source, et celui de droite est la destination :

In [ ]:
lien_gauche = traitlets.dlink((robot.left_motor, 'value'), (slide_gauche, 'value'))
lien_droit = traitlets.dlink((robot.right_motor, 'value'), (slide_droit, 'value'))

# Affiche la boite contenant les slides
display(box_slides)

Si vous bougez les slides, vous ne verrez pas les moteurs tourner... Par contre, si vous exécuter le programme ci-dessous, vous verrez les slides bouger !

In [ ]:
robot.forward(0.3)
time.sleep(1.0)
robot.stop()

### Attacher des fonctions à des évènements

Une autre manière d'utiliser les traitlets est d'attacher des fonctions (comme par exemple la fonction ``forward``) à des évènements.

Ces fonctions seront exécutées lorsqu'un évènement particulier arrivera, et transmettront des informations sur les changement engendrés, comme par exemple une vieille ``(old)`` valeur et une nouvelle ``(new)`` valeur.  

Mais encore une fois, le plus simple est de regarder un exemple... Commençons par créer des boutons qui vont nous permettre de commander le robot :

In [ ]:
# Création des boutons
layout_bouton = widgets.Layout(width='100px', height='80px', align_self='center')

bouton_stop = widgets.Button(description='stop', button_style='danger', layout=layout_bouton)
bouton_avance = widgets.Button(description='forward', layout=layout_bouton)
bouton_recul = widgets.Button(description='backward', layout=layout_bouton)
bouton_gauche = widgets.Button(description='left', layout=layout_bouton)
bouton_droit = widgets.Button(description='right', layout=layout_bouton)

# Affiche les boutons
boite_milieu = widgets.HBox([bouton_gauche, bouton_stop, bouton_droit], layout=widgets.Layout(align_self='center'))
boite_controls = widgets.VBox([bouton_avance, boite_milieu, bouton_recul])
display(boite_controls)

Pour l'instant, ces boutons ne font rien ... Il faut tout d'abord définir les actions à effectuer lorsque les boutons sont appuyés :

In [ ]:
# Action à éxécuter lorsque le bouton stop sera appuyé
def action_stop(change):
    robot.stop()
    
# Action à éxécuter lorsque le bouton d'avance sera appuyé
def action_avance(change):
    robot.forward(0.4)
    time.sleep(0.5)
    robot.stop()

# Action à éxécuter lorsque le bouton de recul sera appuyé
def action_recul(change):
    robot.backward(0.4)
    time.sleep(0.5)
    robot.stop()

# Action à éxécuter lorsque le bouton gauche sera appuyé
def action_gauche(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

# Action à éxécuter lorsque le bouton droit sera appuyé
def action_droit(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

 Ensuite, il suffit de lier les actions définies précédemment aux évènements engendrés lors de l'appui sur les boutons. Ces évènements sont de type ``on_click`` :

In [ ]:
# Lien des actions avec les évènement des boutons
bouton_stop.on_click(action_stop)
bouton_avance.on_click(action_avance)
bouton_recul.on_click(action_recul)
bouton_gauche.on_click(action_gauche)
bouton_droit.on_click(action_droit)

Essayez maintenant d'appuyer sur les boutons !

### Chien de garde

Pour terminer, nous allons voir comment utiliser un "chien de garde" : son rôle va être de surveiller le bon fonctionnement de la connexion entre jupyter et le robot, et de l'arrêter en cas de besoin.

Cette surveillance est exécutée à intervalles réguliers, et vous pouvez modifier la période de surveillance (en secondes) avec le slide.

Si la communication entre jupyter et le robot est défectueuse, l'attribut `status` du chien de garde aura pour nouvelle valeur ``dead``. Dès que la connexion sera restaurée, l'attribut aura pour valeur ``alive``.

Pour créer ce chien de garde, on utilise la classe ``Heartbeat``

In [ ]:
from jetbot import Heartbeat

chien_de_garde = Heartbeat()

# Définition de la fonction à appeler lorsque le chien de garde change de status
# Si le status passe à l'état dead, alors on arrête de le robot
def action_chien_de_garde(change):
    if change['new'] == chien_de_garde.Status.dead:
        robot.stop()
        
# Active l'observation de l'attribut "status" du chien de garde
# Et appel de la fonction "action_chien_de_garde" si il y a un changement
chien_de_garde.observe(action_chien_de_garde, names='status')

# Création d'un slide pour modifier la période d'observation
slide_periode = widgets.FloatSlider(description='period', min=0.001, max=0.5, step=0.01, value=0.5)

# Lien entre le slide et le chien de garde
traitlets.dlink((slide_periode, 'value'), (chien_de_garde, 'period'))

display(slide_periode)

Maintenant que la surveillance est activée, exécuter le code ci-dessous pour actionner le robot. Ensuite, diminuez la période de la surveillance... Vous verrez le robot s'arrêter lorsque la période sera trop petite.

Par exemple, si le robot s'arrête avec une période de 0.03, cela signifie que la liaison entre jupyter et le robot n'a aps pu être validée en moins de 0.03s.

In [ ]:
robot.left(0.2) 